<a href="https://colab.research.google.com/github/ASO1801038/AI_teach2020/blob/master/2020AI0305_fishvideo_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#魚アリ
import urllib.request as req
url = "https://github.com/masatokg/sample_photo/raw/master/fish.zip"
save_file = "fish.zip"
req.urlretrieve(url,save_file)

#魚ナシ
import urllib.request as req
url = "https://github.com/masatokg/sample_photo/raw/master/nofish.zip"
save_file = "nofish.zip"
req.urlretrieve(url,save_file)

#解凍
!unzip -o -q "./fish.zip"
!unzip -o -q "./nofish.zip"

In [6]:
import cv2
import os, glob, pickle
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

image_size = (64,32)
path = "/content"
path_fish = path + "/fish"
path_nofish = "/nofish"
x = []
y = []
print(path_fish)
print(path_nofish)

/content/fish
/nofish


In [8]:
#関数の定義
def read_dir(path, label, x, y, image_size):
  files = glob.glob(path + "/*.jpg")
  print(files)
  for f in files:
    #読込
    img = cv2.imread(f)
    img = cv2.resize(img, image_size)
    img_data = img.reshape(-1)
    x.append(img_data)
    y.append(label)

In [12]:
#画像読込
read_dir(path_nofish, 0, x, y, image_size)
read_dir(path_fish, 1, x, y, image_size)

#分割
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)

#学習モデルを生成、学習
clf = RandomForestClassifier()
clf.fit(x_train,y_train)

#精度確認
y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred))

#ファイル保存
with open("fish.pkl","wb") as fp:
  pickle.dump(clf, fp)

[]
['/content/fish/1254.jpg', '/content/fish/1286.jpg', '/content/fish/1158.jpg', '/content/fish/1231.jpg', '/content/fish/1394.jpg', '/content/fish/274.jpg', '/content/fish/1414.jpg', '/content/fish/2815.jpg', '/content/fish/248.jpg', '/content/fish/1268.jpg', '/content/fish/2863.jpg', '/content/fish/1969.jpg', '/content/fish/170.jpg', '/content/fish/1531.jpg', '/content/fish/223.jpg', '/content/fish/1260.jpg', '/content/fish/1395.jpg', '/content/fish/3029.jpg', '/content/fish/1551.jpg', '/content/fish/1377.jpg', '/content/fish/3193.jpg', '/content/fish/1287.jpg', '/content/fish/1326.jpg', '/content/fish/3798.jpg', '/content/fish/209.jpg', '/content/fish/1996.jpg', '/content/fish/3964.jpg', '/content/fish/4014.jpg', '/content/fish/3769.jpg', '/content/fish/1249.jpg', '/content/fish/1988.jpg', '/content/fish/3763.jpg', '/content/fish/1308.jpg', '/content/fish/2827.jpg', '/content/fish/3030.jpg', '/content/fish/1950.jpg', '/content/fish/1443.jpg', '/content/fish/1247.jpg', '/content/fis

In [19]:
import cv2, os, copy, pickle
from google.colab.patches import cv2_imshow

#学習済モデルを読込
with open("fish.pkl","rb") as fp:
  clf = pickle.load(fp)

output_dir = "./bestshot"
if not os.path.isdir(output_dir): os.mkdir(output_dir)

img_last = None
fish_th = 3
count = 0
frame_count = 0

In [20]:
#動画読込処理
url = "https://github.com/masatokg/sample_photo/raw/master/fish.zip"
save_file = "fish.mp4"
req.urlretrieve(url,save_file)

('fish.mp4', <http.client.HTTPMessage at 0x7f251a972ac8>)

In [30]:
cap = cv2.VideoCapture(save_file)
while True:
  is_ok, frame = cap.read()
  if not is_ok:
    print("end")
    break

frame = cv2.resize(frame, (640,360))
frame2 = copy.copy(frame)
frame_count += 1
gray = cv2.cvtColor(frame,cv2.COROL_BGR2GRAY)
gray = cv2.GaussianBlur(gray,(15,15),0)
img_b = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)[1]

if not img_last is None:
  frame_diff = cv2.absdiff(img_last,img_b)
  cnts = cv2.findContours(frame_diff,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
  fish_count = 0
  for pt in cnts:
    x,y,w,h = cv2.boundingRect(pt)
    if w<100 or w>500:
      continue

    imgex = frame[y:y+h,x:x+w]
    imgex = cv2.resize(imgex,(64,32))
    image_data = imgex.reshape(-1,)

    pred_y = clf.predict([image_data])

    if pred_y[0] == 1:
      fish_count += 1
      cv2.rectangle(frame2,(x,y),(x+w,y+h),(0,255,0),2)
  #syuturyoku
  if fish_count > fish_th:
    fname = output_dir + "/fish" + str(count) + ".jpg"
    cv2.imwrite(fname,frame)
    count += 1

  img_last = img_b

cap.release()
print("ok",count, "/", frame_count, "/枚検出")

end


error: ignored